**Import Libraries**

In [0]:
import json
import pandas as pd

from google.colab import drive

drive.mount('/content/drive/',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


**Import dataset into a DataFrame**

In [0]:
#Change source path of exported data ,in JSON ,  according to directory stored in Google drive
df = pd.read_json('/content/drive/My Drive/ML EFFORT/JSON/result.json')
#df = pd.read_json('/content/drive/My Drive/result_keeve.json')
df1 = pd.DataFrame(df['chats']['list'])

**Display DataFrame**

In [0]:
df1

,name,type,id,messages
0,LUKAS SMU,personal_chat,4875069116,"[{'id': 88306, 'type': 'message', 'date': '201..."
1,Melody Toh,personal_chat,4393281250,"[{'id': 38222, 'type': 'message', 'date': '201..."
2,WU SHEN SMU,personal_chat,4705381681,"[{'id': 20956, 'type': 'message', 'date': '201..."
3,CHEW WEI BIN SMU,personal_chat,4868563798,"[{'id': 129210, 'type': 'message', 'date': '20..."
4,Timothy SMU,personal_chat,4387890922,"[{'id': 32085, 'type': 'service', 'date': '201..."
...,...,...,...,...
358,Chee Yong 🐢,personal_chat,4426696872,"[{'id': 23374, 'type': 'service', 'date': '201..."
359,Deb,personal_chat,4398551961,"[{'id': 590, 'type': 'message', 'date': '2017-..."
360,Dogston,personal_chat,4428462465,"[{'id': 5, 'type': 'message', 'date': '2017-03..."
361,Rinto,personal_chat,4335836381,"[{'id': 158, 'type': 'message', 'date': '2017-..."


**Convert column of relavant datain to a new DataFrame to work on**

In [0]:
df2=df1['messages'].tolist()

In [0]:
new_list=[]
for i in range(len(df2)):
    for j in range(len(df2[i])):
        new_list.append(df2[i][j])
df3=pd.DataFrame(new_list)

**Store reduced dimensions of DataFrame to a temporary variable**

In [0]:

#Remove rows where there is data in unwanted columns (need to add width,heigth etc to preserve emoji)
headersWanted = ['id','type','date','edited','from','from_id','text','reply_to_message_id','width','height','duration_seconds','thumbnail','media_type','sticker_emoji']
tmpDF = df3
for (columnName, columnData) in df3.iteritems():
 # print(len(tmpDF))
  if (columnName not in headersWanted):
      tmpDF =tmpDF[tmpDF[columnName].isna()]
  else:
    continue
#Further filtering
tmpDF =tmpDF[tmpDF.type != 'service']   

#Drop unnecessary columns
for (columnName, columnData) in tmpDF.iteritems():
  if (columnName not in headersWanted):
    #print(columnName)
    tmpDF=tmpDF.drop(columnName, axis=1)
  else:
    continue  

**Display temporary DataFrame with reduced Dimensions**

In [0]:
tmpDF

,id,type,date,edited,from,from_id,text,reply_to_message_id,width,height,duration_seconds,thumbnail,media_type,sticker_emoji
0,88306,message,2019-02-18T12:08:05,1970-01-01T08:00:00,Zhi Poh,346045672.0,Hi lukas,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,88307,message,2019-02-18T12:08:18,1970-01-01T08:00:00,Zhi Poh,346045672.0,Are we presenting tgt this wednesday?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,88308,message,2019-02-18T12:10:10,1970-01-01T08:00:00,LUKAS SMU,580101820.0,Yep!,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,88309,message,2019-02-18T12:10:11,1970-01-01T08:00:00,LUKAS SMU,580101820.0,Er,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,88310,message,2019-02-18T12:10:18,1970-01-01T08:00:00,LUKAS SMU,580101820.0,My slides i just putting pics uh,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165803,26,message,2017-03-08T09:57:16,1970-01-01T08:00:00,Zhi Poh,346045672.0,Still nid to call?,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165804,39,message,2017-03-08T10:13:09,1970-01-01T08:00:00,Jerry NS,17456490.0,O fuck,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165805,40,message,2017-03-08T10:13:10,1970-01-01T08:00:00,Jerry NS,17456490.0,OK nvm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165806,41,message,2017-03-08T10:13:11,1970-01-01T08:00:00,Jerry NS,17456490.0,HAHAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Content concatenation with noise filtering**

In [0]:
keyFrom=""
valueTo=""
receiving = False
firstRow = True
firstRow_key = True
#Change user variable to the string value of the user's name whose data is going to used
user ="Zhi Poh"
output = pd.DataFrame({'Input':[],'Output':[]})

for i in range(len(tmpDF)):
  row = tmpDF.iloc[i].tolist()
  sender = row[4]

  # Skip iteration if the text message is empty (Sticker etc.)
  if (len(row[6])==0):
    continue
  if ("{'type':" in str(row[6])):
    continue

  if (sender == user):
    ## User 
    # Not ready to receive or if text field is empty
    if (not receiving):
      continue

    else:
      ## Start of User's text
      if (firstRow):
        firstRow = False
      else:
        ##Concatenate strings here
        valueTo += " "

      if (len(str(row[6]))==1):
        valueTo+=str(row[6]).lower()
      if (len(str(row[6]))>=2):
        valueTo+=str(row[6])[0].lower()+str(row[6])[1:]
  

  else:
      ## Others
      if (not receiving):
        # on first instance of message from others
        keyFrom = str(row[6])
        firstRow_key = False
        # User is now receving
        receiving = True

      elif (receiving):
        ## Receving State (Sender - User - Sender ....)
        ## Message previously was from user(user message was concatenating), add current info to dict
        if (not firstRow):
            output = output.append(pd.DataFrame({'Input' : [keyFrom], "Output" : [valueTo] }),ignore_index = True)
            # Reset
            firstRow = True
            firstRow_key = True
            valueTo = ""
            keyFrom=""
      


        if (firstRow_key):
            keyFrom+=str(row[6])
            firstRow_key = False
        else:
          keyFrom += " "

          if (len(str(row[6]))==1):
              keyFrom+=str(row[6]).lower()
          else:
              keyFrom+=str(row[6])[0].lower()+str(row[6])[1:]  


if sender == user:
  output = output.append(pd.DataFrame({'Input' : [keyFrom], "Output" : [valueTo] }),ignore_index = True)



In [0]:
output

,Input,Output
0,Yep! er my slides i just putting pics uh maybe...,okie i just cover 4 or 5 points and conclude
1,Ok i set up google slides later when i reach h...,im done le lol
2,Eh where u ah bro,bro.\nIm probably taking 4 IS mods next term\n...
3,Can ah,wbu?
4,I havent decided yet i let u know later,i chose this based on what my senior say he sa...
...,...,...
33989,Hi Zhi Poh..\n\nU can do the payment by bank t...,"okie i am in camp nw, will transfer once out"
33990,Sure thing thanks hi Zhi Poh..am checking if u...,"hi mr rinto, i have passed the training fees t..."
33991,Hi Zhi Poh..we have receive the fees..hope to ...,okie
33992,Bro ask Albert call office to look for me don'...,"bro, he is currently overseas in shanghai stil..."


**Export base preprocessed data to spreadsheet.Change exported file name if needed**

In [0]:
output.to_excel('zhipoh_output.xlsx',index_label=None,index =False)
from google.colab import files
files.download('zhipoh_output.xlsx')

'Haha okie'